<a href="https://colab.research.google.com/github/DaryaTereshchenko/DaryaTereshchenko/blob/main/glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.initializers import Constant

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [ ]:
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter

In [ ]:
data = pd.read_csv("binary_ethos.csv", sep=",", encoding="utf-8")
shuffled = data.sample(frac=1, random_state=300)
shuffled.T

,695,722,531,490,68,444,416,796,918,932,...,505,406,103,91,725,9,714,917,481,209
text,U r still so beautiful and I am still watching...,There's a lot of confusion about male female v...,They don' t clean up the rubbish that they all...,"I dont hate those kids with these problems, bu...",Only black lives matter. fuck white people the...,Those illegals want to sue the company for mon...,France let them refugees in and this what happ...,"My cousin has that, he's 7 but still can't tal...",God bless you dears,Enough is Enough,...,Anyone who can' cope as the gender they are bo...,"Humans with third leg I hope to distinct soon,...",I just don't like Homosexuals raising kids it'...,Indians are shit but burgers only have broken ...,Then the same people choosing the same answer,Why do negroes tear down their own neighborhoo...,They must be really suffering if they're throw...,"God,my humble question to u almighty... Y u se...","If you are a fking queen, you can move in any ...",Lgbt now spread in asia. Many anti lgbt in asi...
lable,0,0,0,0,1,0,1,0,0,0,...,0,1,1,1,0,1,0,0,0,1


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
import regex as re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def clean_text(line):
  stops = set(stopwords.words("english"))
  processed_line = line.translate(str.maketrans('','', string.punctuation))
  processed_line = line.lower().split()
  processed_line = [word for word in processed_line if word not in stops]
  text = " ".join(processed_line)
  return text

In [ ]:
shuffled['text'] = shuffled['text'].map(lambda x: clean_text(x))
print(shuffled['text'])

695    u r still beautiful still watching babe chat l...
722    there's lot confusion male female vs. masculin...
531     don' clean rubbish throw floor place looks mess.
490    dont hate kids problems, im saying feel ver un...
68     black lives matter. fuck white people cancer e...
                             ...                        
9      negroes tear neighborhoods black thug gets kil...
714    must really suffering they're throwing food gr...
917    god,my humble question u almighty... u send ea...
481    fking queen, move direction. please move away ...
209    lgbt spread asia. many anti lgbt asian worried...
Name: text, Length: 988, dtype: object


In [ ]:
train = shuffled.sample(frac=0.8, random_state=20)
train.reset_index(drop=True)

,text,lable
0,"winnie, even tinnie, flat chest would hire mis...",0
1,live youre wanted. deport jews muslims,1
2,fuck homo dumb ass prick! shoulda let get fuck...,1
3,next time like front one like shout left ear t...,0
4,world would better without white people stole ...,1
...,...,...
785,easy seattle wheeler! towns countries!,0
786,bro delete yo shit youtube rn. fuck outta yo c...,0
787,indian ned flanders. look like professionally ...,1
788,beeing gay anti human,1


In [ ]:
test = shuffled.drop(train.index)
test = test.sample(frac=1, random_state=200)
test.reset_index(drop=True)

,text,lable
0,looks like gay poster boy syphilis,1
1,lyrics wrong,0
2,beautiful every way!,0
3,"believe me, whites superior every way nips wes...",1
4,well..congratulations u loose one ur subscribe...,0
...,...,...
193,certainly can. make illegal use them. boom. do...,0
194,regret approaching girl. fuck gynocentric soci...,1
195,fuck sweden. let die like degenerates are!,1
196,honestly hate china. u disagree cant. horrible...,1


In [ ]:
from nltk.tokenize import word_tokenize
def create_corpus(data):
  corpus = []
  for text in train["text"]:
    words = [word for word in word_tokenize(text)]
    corpus.append(words)
  return corpus

In [ ]:
corpus = create_corpus(train)
print(corpus[0])

['winnie', ',', 'even', 'tinnie', ',', 'flat', 'chest', 'would', 'hire', 'miss', '.', 'way', '.']


In [ ]:
for line in corpus:
  for token in line:
    if token in set(string.punctuation):
      line.remove(token)

In [ ]:
print(corpus[10])

['wtf', 'teacher', 'expelled', 'that', "'s", 'way', 'treat', 'disabled', 'kid', 'specially', 'autistic', 'one']


In [ ]:
num_words = len(corpus)
print(num_words)

790


In [ ]:
train_sentences = train.text
train_labels = train["lable"]
train_sentences[10]

'gives aa fuck shit diamonds jews fuck'

In [ ]:
test_sentences = test.text
test_labels = test["lable"]
test_sentences[207]

'looks like gay poster boy syphilis'

In [ ]:
max_len = 50

In [ ]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=max_len, truncating="post", padding="post")

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding="post", truncating="post")

In [ ]:
print(train.text[10])
print(train_sequences[10])

gives aa fuck shit diamonds jews fuck
[407, 138, 48, 322, 100, 101, 115, 10]


In [ ]:
word_index = tokenizer.word_index
print("Number of unique words: ", len(word_index))

Number of unique words:  3254


In [ ]:
print(test_sequences[10])
print(test_padded[10])

[138, 526, 455, 42, 10, 73, 318, 23, 36, 45, 336]
[138 526 455  42  10  73 318  23  36  45 336   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [ ]:
word_index["clearly"]

525

In [ ]:
# Create the Embedding dictionary

In [ ]:
embedding_dict = {}
with open("glove.twitter.27B.100d.txt", "r") as f:
  for line in f:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:], "float32")
    embedding_dict[word] = vectors
  f.close()

In [ ]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 100))

for word, i in word_index.items():
  if i < num_words:
    emb_vec = embedding_dict.get(word)
    if emb_vec is not None:
      embedding_matrix[i] = emb_vec

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.3515    ,  0.16734999, -0.32128999, ..., -0.65280998,
         0.70542002,  0.41204   ],
       [-0.21063   , -0.010992  , -0.17552   , ..., -0.37547001,
         0.58029002,  0.16067   ],
       ...,
       [ 0.022097  , -0.24957   ,  0.14702   , ...,  0.29541001,
        -0.066459  , -0.13369   ],
       [ 0.34681001, -0.32754999, -0.74555999, ..., -0.59878999,
         0.12052   , -0.19338   ],
       [-0.13657001, -0.18486001, -0.10945   , ..., -0.79948002,
         0.44027001, -0.68939   ]])

In [ ]:
word_index["faggot"]

290

In [ ]:
embedding_dict.get("faggot").shape

(100,)

In [ ]:
(embedding_matrix[290] == embedding_dict.get("faggot")).all()

True

In [ ]:
print(train_padded.shape)
print(train_labels.shape)

(790, 50)
(790,)


In [ ]:
print(test_padded.shape)
print(test_labels.shape)

(198, 50)
(198,)


**Baseline Model with** **GloVe**

In [ ]:
model = Sequential()

model.add(
    Embedding(
        num_words,
        100,
        embeddings_initializer=Constant(embedding_matrix),
        input_length=max_len,
        trainable=False,
    )
)

model.add(LSTM(100, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))

optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history = model.fit(
    train_padded,
    train_labels,
    epochs=20,
    validation_data=(test_padded, test_labels),
    verbose=1,
)

Epoch 1/20
25/25 [==============================] - 2s 27ms/step - loss: 0.6916 - accuracy: 0.5658 - val_loss: 0.6904 - val_accuracy: 0.5455
Epoch 2/20
25/25 [==============================] - 0s 10ms/step - loss: 0.6854 - accuracy: 0.5709 - val_loss: 0.6899 - val_accuracy: 0.5455
Epoch 3/20
25/25 [==============================] - 0s 8ms/step - loss: 0.6825 - accuracy: 0.5684 - val_loss: 0.6902 - val_accuracy: 0.5455
Epoch 4/20
25/25 [==============================] - 0s 8ms/step - loss: 0.6807 - accuracy: 0.5709 - val_loss: 0.6912 - val_accuracy: 0.5455
Epoch 5/20
25/25 [==============================] - 0s 10ms/step - loss: 0.6577 - accuracy: 0.5810 - val_loss: 0.6418 - val_accuracy: 0.6061
Epoch 6/20
25/25 [==============================] - 0s 8ms/step - loss: 0.5996 - accuracy: 0.6924 - val_loss: 0.6114 - val_accuracy: 0.6818
Epoch 7/20
25/25 [==============================] - 0s 8ms/step - loss: 0.5635 - accuracy: 0.7063 - val_loss: 0.6034 - val_accuracy: 0.6616
Epoch 8/20
25/25 

In [ ]:
sequences = test_sequences
padded = test_padded

In [ ]:
pred = model.predict(padded)
pred_int = pred.round().astype("int")

7/7 [==============================] - 0s 4ms/step


In [ ]:
from sklearn.metrics import make_scorer, classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
print("=== Classification Report ===")
print(classification_report(test_labels, pred_int))
print('\n')

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.78      0.63      0.70       108
           1       0.64      0.79      0.71        90

    accuracy                           0.70       198
   macro avg       0.71      0.71      0.70       198
weighted avg       0.72      0.70      0.70       198





In [ ]:
pred_int

In [ ]:
padded[10]

array([ 3,  1, 20, 81,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [ ]:
model.predict(padded[10].reshape(1, -1))

1/1 [==============================] - 0s 19ms/step


array([[0.2855408]], dtype=float32)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
def decode(text):
  return " ".join([reverse_word_index.get(i, "?") for i in text])

In [ ]:
decode(sequences[10])

'white people shit human'

In [ ]:
from sklearn.metrics import make_scorer, classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
print("=== Classification Report ===")
print(classification_report(y_test, y_pred))
print('\n')